In [44]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from scipy.stats import norm
import math
from numpy.linalg import inv

%matplotlib inline

In [45]:
plt.rcParams['figure.figsize'] = 20,16
mpl.style.use('ggplot')
#data = pd.read_csv('./test1_Standard_Result.csv')
data = pd.read_csv('./preliminary.csv')
validation = pd.read_csv('./test1_Standard_Data.csv')
data_range = data.UTC_odom.size
valid_range = validation.UTC.size
R = 6378137
GPS_y = 319.632/1000
imu_y = -53.398/1000

overlap = []
for n in range(1,data.UTC_odom.size):
    if data.UTC_odom[n] == data.UTC_odom[n-1]:
        overlap.append(n)
data.drop(data.index[overlap],inplace = True)
data = data.reset_index(drop = True)
data.head()
print(data_range)

15234


In [46]:
def sin(x):
    return math.sin(math.radians(x))
def cos(x):
    return math.cos(math.radians(x))
def deg(x):
    return x*360/(2*3.14159265357)

def align_UTC(validation,data):
    ids = []
    ids.append(0)
    for i in range(1,data.UTC_odom.size):
        utc = data.UTC_odom[i]
        if utc < validation.UTC[ids[i-1]]:
            ids.append(ids[i-1])
        else:
            for j in range(ids[i-1],validation.UTC.size-1):
                if utc>validation.UTC[j] and utc<validation.UTC[j+1]:
                    if abs(utc-validation.UTC[j])<abs(utc-validation.UTC[j+1]):
                        ids.append(j)
                    else:
                        ids.append(j+1)
                    break
    return ids            
    
def loss(result,data,validation,val_list):
    loss = 0
    loss_1 = 0
    loss_2 = 0
    loss_3 = 0
    R = 6378137
    GPS_y = -53.398/1000
    GPS_x = 319.632/1000
    for d in range(0,data.UTC_odom.size):    ##regard the odom utc as reference
        x_r = result[d][0] - math.degrees(GPS_y/R*cos(result[d][2])) - math.degrees(GPS_x/R*cos(result[d][2]+90))
        y_r = result[d][1] - math.degrees(GPS_y/R*sin(result[d][2])) - math.degrees(GPS_x/R*sin(result[d][2]+90))
        h_r = result[d][2]
        loss_1 = loss_1 + pow(abs(x_r-validation.Longitude[val_list[d]]),2)
        loss_2 = loss_2 + pow(abs(y_r-validation.Latitude[val_list[d]]),2)
        loss_3 = loss_3 + pow(abs(h_r-validation.Heading[val_list[d]]),2)
        print(pow(abs(x_r-validation.Longitude[val_list[d]]),2),pow(abs(y_r-validation.Latitude[val_list[d]]),2),pow(abs(h_r-validation.Heading[val_list[d]]),2))
    loss = 0.4*(loss_1/data.UTC_odom.size)**0.5/0.000002+0.4*(loss_2/data.UTC_odom.size)**0.5/0.000002+0.2*(loss_3)**0.5/3
    return loss

def observe(i,obs):
    R = 6378137
    y_imu = 825.132/1000
    theta = obs.heading[i]
    vx = obs.vel[i]*cos(theta)
    vy = obs.vel[i]*sin(theta)
    w_2 = obs.ang_z[i]*obs.ang_z[i]+obs.ang_x[i]*obs.ang_x[i]
    ax = (obs.acc_y[i]+w_2*y_imu)*cos(theta)+obs.acc_x[i]*cos(theta+90)
    ay = (obs.acc_y[i]+w_2*y_imu)*sin(theta)+obs.acc_x[i]*sin(theta+90)
    w = obs.ang_z[i]
    Pose = np.array([obs.Longitude[i],
                    obs.Latitude[i],
                    theta,
                    deg(vx/R),
                    deg(vy/R),
                    deg(ax/R),
                    deg(ay/R),
                    w ])
    return Pose

def A_(t):
    A=np.array([[1, 0, 0, t, 0, 0.5*t*t, 0,    0],   #x
                [0, 1, 0, 0, t,    0, 0.5*t*t, 0],   #y
                [0, 0, 1, 0, 0,    0,    0,    t],   #theta
                [0, 0, 0, 1, 0,    t,    0,    0],   #vx
                [0, 0, 0, 0, 1,    0,    t,    0],   #vy
                [0, 0, 0, 0, 0,    1,    0,    0],   #ax
                [0, 0, 0, 0, 0,    0,    1,    0],   #ay
                [0, 0, 0, 0, 0,    0,    0,    1]])  #w
               # x  y  o  vx vy    ax    ay    z
    return A

def generate_result(Pose_real,data):
    result_t0 = 1542356255.054155
    R = 6378137
    GPS_y = -53.398/1000
    GPS_x = 319.632/1000
    length = int(data.UTC_odom[len(Pose_real)-1] - data.UTC_odom[0])*20-10
    with open('./Result.csv', 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        for f in range(0,length):
            t_re = result_t0+0.05*f
            fs = 0
            for d in range(fs,len(Pose_real)):
                if t_re<data.UTC_odom[d] and t_re>data.UTC_odom[d-1]:
                    fs = d
                    ratio = (data.UTC_odom[d]-tr)/(data.UTC_odom[d]-data.UTC_odom[d-1])
                    x_r = Pose_real[d-1][0]*ratio+Pose_real[d][0]*(1-ratio) - math.degrees(GPS_y/R*cos(Pose_real[d][2])) - math.degrees(GPS_x/R*cos(Pose_real[d][2]+90))
                    y_r = Pose_real[d-1][1]*ratio+Pose_real[d][1]*(1-ratio) - math.degrees(GPS_y/R*sin(Pose_real[d][2])) - math.degrees(GPS_x/R*sin(Pose_real[d][2]+90))
                    h_r = Pose_real[d-1][2]*ratio+Pose_real[d][2]*(1-ratio)
                    datain = (t_re,x_r,y_r,h_r)
                    csv_writer.writerow(datain)
                    break
        
    
#validation.head()
data.head()
#data.info()
#validation.info()

,UTC_GPS,Latitude,Longitude,altitude,heading,UTC_imu,ang_x,ang_y,ang_z,acc_x,acc_y,acc_z,UTC_odom,vel
0,1.542356e+09,39.791748,116.498636,22.100998,-144.007004,1.542356e+09,0.000351,-0.00028,0.00024,-0.107287,-0.106012,9.828548,1.542356e+09,0.0


In [47]:
t=0
P_pre=np.eye(8)
P=np.eye(8)
Q=0.0000001*np.eye(8)
R=0.0000001*np.eye(8)
Pose_pre=np.zeros(8)
Pose_back = observe(0,data)   #trust the first observe
Pose_real=[Pose_back]

for i in range(1,data_range):
    t = data.UTC_odom[i] - data.UTC_odom[i-1]
    A = A_(t)
    P_pre=np.dot(np.dot(A,P),A.T)+Q
    Pose_pre = np.dot(A,Pose_back)
    
    K=np.dot(P_pre,inv(P_pre+R))
    obser = observe(i,data)
    if data.UTC_GPS[i]==data.UTC_GPS[i-1]:
        obser[0] = Pose_pre[0]
        obser[1] = Pose_pre[1]
        obser[2] = Pose_pre[2]
    Pose_back=Pose_pre+np.dot(K,obser-Pose_pre)
    P=np.dot(np.eye(8)-K,P_pre)
    Pose_real.append(Pose_back)
# val_list = align_UTC(validation,data)
# print(loss(Pose_real,data,validation,val_list))


KeyError: 1

In [41]:
print(Pose_real)

[array([ 1.16498636e+02,  3.97917481e+01, -1.44007004e+02, -0.00000000e+00,
       -0.00000000e+00,  2.04114564e-07,  1.33945162e-06,  2.39800000e-04])]


In [39]:
import csv
import os
print( int(data.UTC_odom[len(Pose_real)-1] - data.UTC_odom[0])*20-10)
generate_result(Pose_real,data)

-10
